In [1]:
import gym
import ray

In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [3]:
# Initialize Ray
ray.shutdown()
ray.init(ignore_reinit_error=True,num_cpus=8, num_gpus=1)

2020-09-27 15:04:27,282	INFO resource_spec.py:231 -- Starting Ray with 14.6 GiB memory available for workers and up to 7.32 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-27 15:04:27,811	INFO services.py:1193 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.7.73',
 'raylet_ip_address': '192.168.7.73',
 'redis_address': '192.168.7.73:43690',
 'object_store_address': '/tmp/ray/session_2020-09-27_15-04-27_280893_34064/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-27_15-04-27_280893_34064/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-09-27_15-04-27_280893_34064'}

In [ ]:
# https://docs.ray.io/en/latest/tune/user-guide.html#parallelism-gpus
ray.cluster_resources()

In [4]:
print("Dashboard URL: http://{}".format(ray.get_webui_url()))

Dashboard URL: http://localhost:8266


### Register MineRL Gym Environment to RLlib

In [5]:
from minerl_rllib.envs import register

/home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
register()

Registering Environment Name: MineRLTreechopVectorObf-v0
Registering Environment Name: MineRLNavigateVectorObf-v0
Registering Environment Name: MineRLNavigateExtremeVectorObf-v0
Registering Environment Name: MineRLNavigateDenseVectorObf-v0
Registering Environment Name: MineRLNavigateExtremeDenseVectorObf-v0
Registering Environment Name: MineRLObtainDiamondVectorObf-v0
Registering Environment Name: MineRLObtainDiamondDenseVectorObf-v0
Registering Environment Name: MineRLObtainIronPickaxeVectorObf-v0
Registering Environment Name: MineRLObtainIronPickaxeDenseVectorObf-v0
Registering Environment Name: MineRLObtainDiamondSurvivalVectorObf-v0


In [7]:
from ray import tune
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples

tune.run("PPO",
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker": 1,
                 "eager": False,
                 "use_pytorch": True,
#                  'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 }
         ,stop={"training_iteration": 2})  

Instructions for updating:
non-resource variables are not supported in the long term


Trial name,status,loc
PPO_MineRLNavigateDenseVectorObf-v0_6bd8b_00000,RUNNING,


(pid=34143) WARNING:tensorflow:From /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=34143) Instructions for updating:
(pid=34143) non-resource variables are not supported in the long term
(pid=34143) /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=34143)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=34143) 2020-09-27 15:04:34,687	WARNING deprecation.py:30 -- DeprecationWarning: `use_pytorch` has been deprecated. Use `framework=torch` instead. This will raise an error in the future!
(pid=34143) 2020-09-27 15:04:34,687	WARNING deprecation.py:30 -- DeprecationWarning: `eager` has been deprecated. Use `framework=tfe` instead. This will raise an error in the future!


Result for PPO_MineRLNavigateDenseVectorObf-v0_6bd8b_00000:
  custom_metrics: {}
  date: 2020-09-27_15-06-57
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: d397c7a02cba4717abb3c21fd73a4dd0
  experiment_tag: '0'
  hostname: blackbox
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.2
        cur_lr: 5.0e-05
        entropy: 90.78745532035828
        entropy_coeff: 0.0
        kl: 0.037070594262331724
        policy_loss: -0.07416248507797718
        total_loss: 0.20093210937920958
        vf_explained_var: 0.02457410655915737
        vf_loss: 0.26768047269433737
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
  node_ip: 192.168.7.73
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 33.69090909090909
    ram_util_percent: 56.35064935064935

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_6bd8b_00000,RUNNING,192.168.7.73:34143,1,107.321,4000,nan


...

Result for PPO_MineRLNavigateDenseVectorObf-v0_6bd8b_00000:
  custom_metrics: {}
  date: 2020-09-27_15-08-13
  done: true
  episode_len_mean: 6000.0
  episode_reward_max: -5.587623596191406
  episode_reward_mean: -5.587623596191406
  episode_reward_min: -5.587623596191406
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: d397c7a02cba4717abb3c21fd73a4dd0
  experiment_tag: '0'
  hostname: blackbox
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.30000000000000004
        cur_lr: 5.0e-05
        entropy: 90.86579465866089
        entropy_coeff: 0.0
        kl: 0.03146955702686682
        policy_loss: -0.05248393851798028
        total_loss: 0.20206350344233215
        vf_explained_var: 0.026143575087189674
        vf_loss: 0.2451065699569881
    num_steps_sampled: 8000
    num_steps_trained: 8000
  iterations_since_restore: 2
  node_ip: 192.168.7.73
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent:

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_6bd8b_00000,TERMINATED,,2,183.124,8000,-5.58762


Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_6bd8b_00000,TERMINATED,,2,183.124,8000,-5.58762


(pid=34143) *** Aborted at 1601244496 (unix time) try "date -d @1601244496" if you are using GNU date ***
(pid=34143) PC: @                0x0 (unknown)
(pid=34143) *** SIGSEGV (@0x7f89817fa9d0) received by PID 34491 (TID 0x7f89a44b6740) from PID 18446744071587211728; stack trace: ***
(pid=34143)     @     0x7f89a48273c0 (unknown)
(pid=34143)     @     0x7f89a481caab __pthread_clockjoin_ex
(pid=34143)     @     0x7f89a23f22d3 std::thread::join()


(pid=34143)     @     0x7f89a2923493 ray::gcs::GlobalStateAccessor::Disconnect()
(pid=34143)     @     0x7f89a27c2fbc __pyx_pw_3ray_7_raylet_19GlobalStateAccessor_5disconnect()
(pid=34143)     @     0x55e93d0d9b71 _PyMethodDef_RawFastCallKeywords
(pid=34143)     @     0x55e93d0e0aef _PyMethodDescr_FastCallKeywords
(pid=34143)     @     0x55e93d14537c _PyEval_EvalFrameDefault
(pid=34143)     @     0x55e93d0d920b _PyFunction_FastCallKeywords
(pid=34143)     @     0x55e93d140e70 _PyEval_EvalFrameDefault
(pid=34143)     @     0x55e93d0892b9 _PyEval_EvalCodeWithName
(pid=34143)     @     0x55e93d0d9435 _PyFunction_FastCallKeywords
(pid=34143)     @     0x55e93d140be6 _PyEval_EvalFrameDefault
(pid=34143)     @     0x55e93d0892b9 _PyEval_EvalCodeWithName
(pid=34143)     @     0x55e93d08a3e5 _PyFunction_FastCallDict
(pid=34143)     @     0x55e93d0eec70 PyErr_CheckSignals
(pid=34143)     @     0x55e93d0fa9c6 time_sleep
(pid=34143)     @     0x55e93d0d9abd _PyMethodDef_RawFastCallKeywords
(pid=3